# Data collection

Here we scrape the ad library of facebbok to collect all the ads publish by political parties in Belgium.
We only collect : 
* The date of the publication of the ad
* The status of the ad (active or inactive)
* The text of the add
* The url of the photo attached to the ad if any:

## Further improvement
Further improvement includes : 
* For each add collecting:
    * The number of views
    * The budget
    * Who watch the ad
* Collect the text information present on the image attached.
* Collect post of the page
* Collect ads from related pages (local branch of the parti)

In [1]:
import numpy as np
import pandas as pd
import time
import os
from datetime import datetime
from tqdm import tqdm

from selenium import webdriver                    
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

We have do this code for each partie

In [31]:
URL = 'https://www.facebook.com/ads/library/?active_status=all&ad_type=all&country=BE&impression_search_field=has_impressions_lifetime&view_all_page_id=52997544531'  # Define URL
chrome_options = Options()
#chrome_options.add_argument("--headless")
browser = webdriver.Chrome(options=chrome_options)
browser.get(URL)

In [88]:
for j in range(20):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

In [89]:
pubs = browser.find_elements_by_class_name('_7owt')
print(len(pubs))

536


In [90]:
pubs_list = []
for i, pub in tqdm(enumerate(pubs)):
    soup = BeautifulSoup(pub.get_attribute('innerHTML'), 'html.parser')
    pub_dict = {}
    pub_dict['date'] = soup.find(class_="_7jwu").text
    pub_dict['status'] = soup.find(class_="_7jw1").text
    pub_dict['id'] = soup.find(class_="_4rhp").text
    
    text = soup.find(class_="_7jyr")
    if text:
        pub_dict['text'] = text.text
    
    image = soup.find(class_="_7jys")
    if image:
        pub_dict['photo'] = image['src']
    #pub_dict['montant'] = ''
    #pub_dict['vues'] = ''
    pubs_list.append(pub_dict)
    
pubs_df = pd.DataFrame.from_dict(pubs_list)

536it [00:06, 77.15it/s]


In [91]:
pubs_df

,date,status,id,text,photo
0,Début de diffusion le 8 jan 2020,Inactive,ID : 2876702479058376,Voor de eerste keer ooit leggen onze poetshulp...,https://scontent-bru2-1.xx.fbcdn.net/v/t1.6435...
1,Début de diffusion le 9 jan 2020,Actif,ID : 2457694537881187,Tussen Kerst en nieuw kregen sociale huurders ...,NaN
2,Début de diffusion le 9 jan 2020,Inactive,ID : 501603640486772,Poetshulpen zijn terecht kwaad. Zij krijgen ge...,NaN
3,Début de diffusion le 30 déc 2019,Inactive,ID : 456187221695598,Net zoals mensenrechten horen ook dierenrechte...,https://scontent-bru2-1.xx.fbcdn.net/v/t39.168...
4,Début de diffusion le 30 déc 2019,Inactive,ID : 579650385944580,Je beste vriend meenemen naar het ruisthuis. W...,https://scontent-bru2-1.xx.fbcdn.net/v/t39.168...
...,...,...,...,...,...
531,10 avr 2019 - 30 avr 2019,Inactive,ID : 641190196320619,Natuurlijk zijn ons pensioenen betaalbaar. Als...,https://scontent-bru2-1.xx.fbcdn.net/v/t39.168...
532,17 avr 2019 - 20 avr 2019,Inactive,ID : 517032982159660,✋🏼Halt aan de Homans-heffing! 💶 50.000 social...,https://scontent-bru2-1.xx.fbcdn.net/v/t39.168...
533,10 avr 2019 - 30 avr 2019,Inactive,ID : 418234628740515,In ons land zijn er 230.000 mensen die werken ...,https://scontent-bru2-1.xx.fbcdn.net/v/t39.168...
534,9 avr 2019 - 22 avr 2019,Inactive,ID : 2650259395015062,In ons land zijn er 230.000 mensen die werken ...,https://scontent-bru2-1.xx.fbcdn.net/v/t39.168...


In [92]:
pubs_df.to_csv('spa.csv')